In [1]:
import pandas as pd

### State Variables and Defaults

In [49]:
CERT_NUMBER = "Cert_number"
CERT_NAME = "Cert_name"
PATH = "Path"
PATH_RECERT = "Path_recert"
COURSE_CODE = "Course_code"
COURSE_TITLE = "Course_title"
COURSE_VERSION = "Course_version"
TARGET_DAYS = "Target_days"
EXPIRES_IN = "Expires_in"
CERT_DESC = "Cert_desc"

VERSION = "V0"
DOMAIN = "CNL Global"
AVAILABLE_FROM = "1900-01-01"
STATUS = "in effect"
DISPLAY_TO_LEARNER = True
EXPIRY_TYPE = 1
NOTIFY_BEFORE = 90
ENFORCE_TARGET_DATE = False
ONE_CLICK_REG = True

MODULE_REQUIRED = True


### Helper Classes

In [55]:
class Certification:
    def __init__(self, cert_name, target_days, expires_days, description):
        self.cert_name = cert_name
        self.target_days = target_days
        self.expires_days = expires_days
        self.description = description
        self.paths = []
    def add_path(self, path):
        self.paths.append(path)
    def print_info(self):
        print("Cert Name:", self.cert_name)
        print("Paths: ", len(self.paths))
        print("Target: ", self.target_days)
        print("Expires: ", self.expires_days)

class Path:
    def __init__(self, path_name, is_recert):
        self.path_name = path_name
        self.is_recert = is_recert
        self.modules = []
    def add_module(self, module):
        self.modules.append(module)

class Module:
    def __init__(self):
        self.courses = []
    def add_course(self, code, title, version):
        self.courses.append((code,title,version))
    def print_info(self):
        print(self.courses)


### Load Input File

In [28]:
source = pd.read_excel("Input.xlsx")
source.head()

,Cert_number,Cert_name,Target_days,Expires_in,Cert_desc,Path,Path_recert,Module,Course_code,Course_title,Course_version
0,1,Aerial Work Platform Practical - Articulating ...,42,1095,This certification qualifies workers to operat...,Main,False,Main,OSH-1003-Online,Aerial Work Platform Theory (R3),1.6
1,1,Aerial Work Platform Practical - Articulating ...,42,1095,This certification qualifies workers to operat...,Main,False,Main,OSH-3003-A,Aerial Work Platform Practical - Articulating ...,1.0
2,1,Aerial Work Platform Practical - Articulating ...,42,1095,This certification qualifies workers to operat...,Recertification,True,Main,OSH-1003-Online,Aerial Work Platform Theory (R3),1.6
3,1,Aerial Work Platform Practical - Articulating ...,42,1095,This certification qualifies workers to operat...,Recertification,True,Main,OSH-3003-A,Aerial Work Platform Practical - Articulating ...,1.0
4,2,Aerial Work Platform Practical - Articulating ...,42,1095,This certification qualifies workers to operat...,Main,False,Main,OSH-1003-Online,Aerial Work Platform Theory (R3),1.6


### Set Variables

In [56]:
## Read Certs into separate df
df_list = [d for _, d in source.groupby([CERT_NUMBER])]

certs = []
## For each cert, generate paths
for cert in df_list:
    cert_name = cert[CERT_NAME].iloc[0]
    cert_target = cert[TARGET_DAYS].iloc[0]
    cert_exp = cert[EXPIRES_IN].iloc[0]
    cert_des = cert[CERT_DESC].iloc[0]
    #print(cert_name, cert_target, cert_exp, cert_des)
    curr_cert = Certification(cert_name, cert_target, cert_exp, cert_des)

    ## Read paths for the cert
    paths = []
    path_list = [d for _, d in cert.groupby([PATH])]
    ## For each path, generate list of courses and module
    for path in path_list:
        mod = Module()
        for _, row in path.iterrows():
            course_code = row[COURSE_CODE]
            course_title = row[COURSE_TITLE]
            course_version = row[COURSE_VERSION]
            #print(course_code, course_title, course_version)
            mod.add_course(course_code, course_title, course_version)
            #mod.print_info()
        path_name = path[PATH].iloc[0]
        path_recert = path[PATH_RECERT].iloc[0]
        #print(path_name, path_recert)
        curr_path = Path(path_name, path_recert)
        curr_cert.add_path(path)
    certs.append(curr_cert)

for cert in certs:
    cert.print_info()


Cert Name: Aerial Work Platform Practical - Articulating Boom 135 ft
Paths:  2
Target:  42
Expires:  1095
Cert Name: Aerial Work Platform Practical - Articulating Boom 60 ft or Less
Paths:  2
Target:  42
Expires:  1095


### Write Output